In [2]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp
%matplotlib inline

import findspark
findspark.init('/opt/spark/spark-2.3.2-bin-hadoop2.7')

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
data = spark.read.option("delimiter", "\t").option("header", "true").csv("../Project/en.openfoodfacts.org.products.csv") 

In [4]:
data.dtypes

[('code', 'string'),
 ('url', 'string'),
 ('creator', 'string'),
 ('created_t', 'string'),
 ('created_datetime', 'string'),
 ('last_modified_t', 'string'),
 ('last_modified_datetime', 'string'),
 ('product_name', 'string'),
 ('generic_name', 'string'),
 ('quantity', 'string'),
 ('packaging', 'string'),
 ('packaging_tags', 'string'),
 ('brands', 'string'),
 ('brands_tags', 'string'),
 ('categories', 'string'),
 ('categories_tags', 'string'),
 ('categories_en', 'string'),
 ('origins', 'string'),
 ('origins_tags', 'string'),
 ('manufacturing_places', 'string'),
 ('manufacturing_places_tags', 'string'),
 ('labels', 'string'),
 ('labels_tags', 'string'),
 ('labels_en', 'string'),
 ('emb_codes', 'string'),
 ('emb_codes_tags', 'string'),
 ('first_packaging_code_geo', 'string'),
 ('cities', 'string'),
 ('cities_tags', 'string'),
 ('purchase_places', 'string'),
 ('stores', 'string'),
 ('countries', 'string'),
 ('countries_tags', 'string'),
 ('countries_en', 'string'),
 ('ingredients_text', 'str

In [19]:
data.count()

684106

In [28]:
France = data.filter(data["countries"]=="France")
France.filter(France.origins != "").count()

23748

In [36]:
origins = France.groupBy("origins").count()
origins.show()

+--------------------+-----+
|             origins|count|
+--------------------+-----+
|       Côte d'Ivoire|   22|
|       Non spécifiée|    8|
|Côtes du Rhône,Fr...|    1|
|France,Pêcheries ...|    1|
|   France ou Espagne|    2|
|Vin,Bourgogne,France|    1|
|Noix de Saint-Jac...|    1|
|     basse Normandie|    1|
|      Porc de France|    1|
|Languedoc Roussil...|    1|
|     France,Bordeaux|    2|
|Carpe origine France|    1|
| Beaumont-de-Lomagne|    1|
|                 Mer|    1|
|farine de blé T18...|    1|
|           France,35|    1|
|      Halluin,France|    1|
|Pérou,Coopérative...|    5|
|Chine,Privince du...|    1|
|La Réunion,Morbih...|    1|
+--------------------+-----+
only showing top 20 rows



In [42]:
origins.sort(origins["count"].desc()).show()

+--------------------+------+
|             origins| count|
+--------------------+------+
|                null|152744|
|              France|  8938|
|    Union Européenne|  1303|
|             Espagne|   696|
|              Italie|   652|
|    Union européenne|   363|
|  France,Hors France|   178|
|             Norvège|   175|
|               Pérou|   171|
|               Maroc|   156|
|               Chine|   143|
|             Turquie|   138|
|           Allemagne|   126|
|            Belgique|   116|
|           Thaïlande|   116|
|    Sud-Ouest,France|   115|
|France,Union Euro...|   113|
|              france|   107|
|               Grèce|   105|
|     France,Bretagne|    83|
+--------------------+------+
only showing top 20 rows



In [49]:
MP = France.filter(France.manufacturing_places != "")
MP.count()

37083

In [50]:
FP = France.filter(France.first_packaging_code_geo != "")
FP.count()

22806

In [51]:
PP = France.filter(France.purchase_places != "")
PP.count()

56475

In [67]:
O = France.filter(France.origins != "")

In [68]:
join = MP.join(FP, ["code"], "inner").join(PP, ["code"], "inner").join(O,["code"], "inner")
#join.select("code", "manufacturing_place", "first_packaging_code_geo", "purchase_places").show()
join.count()

5633